In [1]:
import pandas as pd
import json

In [2]:
# Load JSON Data
with open('data/shp/patchcrop-sensor-mapping.json') as f:
    data = json.load(f)

# Parse JSON Object
sensor_mapping = data['sensor-mapping']
parsed_data = []

for patch, attributes in sensor_mapping.items():
    if patch.startswith('patch') and not patch.endswith('mobil'):
        left_lon = attributes['position']['left']['long']
        right_lon = attributes['position']['right']['long']
        left_lat = attributes['position']['left']['lat']
        right_lat = attributes['position']['right']['lat']
        parsed_data.append([patch, left_lon, left_lat, right_lon, right_lat])

# Organize into df
columns = ['patch', 'l_lon', 'l_lat','r_lon', 'r_lat']
coords = pd.DataFrame(parsed_data, columns=columns)
coords['patch'] = coords['patch'].str.replace('patch_', '')
coords['lon'] = coords[['l_lon', 'r_lon']].mean(axis=1)
coords['lat'] = coords[['l_lat', 'r_lat']].mean(axis=1)
# coords = coords.drop(columns = ['l_lon', 'l_lat','r_lon', 'r_lat'])

coords.to_csv('data/shp/patch_coords.csv', index=False)

In [3]:
# Merge with nmin df
nmin = pd.read_csv('data/true_data/nmin.csv')
coords['patch'] = coords['patch'].astype('int64')
merge = pd.merge(nmin, coords, on='patch', how='left')
merge

,patch,date,depth,NH4,DIF,gwc,BD,mg.N.NH4.kg.dry.soil,mg.NO3.N..kg.dry.soil,kgNO3,...,kgNmin,X.1,X.3,vwc,l_lon,l_lat,r_lon,r_lat,lon,lat
0,12.0,2020-08-04,30.0,0.025667,0.166667,4.836667,1.6,0.266667,2.102667,10.093333,...,11.373333,NaN,NaN,7.738667,14.134927,52.446445,14.134927,52.446444,14.134927,52.446444
1,12.0,2020-08-04,60.0,0.031000,0.000000,2.890000,1.7,0.323333,0.335333,1.706667,...,3.343333,NaN,NaN,4.913000,14.134927,52.446445,14.134927,52.446444,14.134927,52.446444
2,12.0,2020-08-04,90.0,0.020000,0.000000,5.720000,1.7,0.213333,0.227667,1.160000,...,2.236667,NaN,NaN,9.724000,14.134927,52.446445,14.134927,52.446444,14.134927,52.446444
3,12.0,2020-11-11,30.0,0.097000,0.033333,14.116667,1.6,1.130000,1.582333,7.593333,...,13.020000,NaN,NaN,22.586667,14.134927,52.446445,14.134927,52.446444,14.134927,52.446444
4,12.0,2020-11-11,60.0,0.026667,0.000000,11.320000,1.7,0.306667,0.398333,2.030000,...,3.573333,NaN,NaN,19.244000,14.134927,52.446445,14.134927,52.446444,14.134927,52.446444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,119.0,2022-12-09,90.0,0.021667,0.000000,4.603333,1.7,0.226667,0.429333,2.190000,...,3.350000,NaN,NaN,7.825667,14.150519,52.448894,14.150467,52.448891,14.150493,52.448892
782,119.0,2023-02-24,30.0,0.132667,-0.066667,9.943333,1.6,1.476667,0.793333,3.806667,...,10.886667,NaN,NaN,15.909333,14.150519,52.448894,14.150467,52.448891,14.150493,52.448892
783,119.0,2023-02-24,60.0,0.035000,0.000000,6.133333,1.7,0.370000,0.267333,1.366667,...,3.260000,NaN,NaN,10.426667,14.150519,52.448894,14.150467,52.448891,14.150493,52.448892
784,119.0,2023-02-24,90.0,0.037667,0.033333,7.720000,1.7,0.406667,0.622333,3.173333,...,5.250000,NaN,NaN,13.124000,14.150519,52.448894,14.150467,52.448891,14.150493,52.448892
